In [1]:
import pandas as pd
import numpy as np
#path = r'C:\Users\user/'

color_layout_features =pd.read_pickle("color_layout_descriptor.pkl")
bow_surf  = pd.read_pickle("bow_surf.pkl")
color_hist_features  = pd.read_pickle("hist.pkl")
labels  = pd.read_pickle("labels.pkl")

#color_layout_features  = pd.read_pickle(path + "/color_layout_descriptor.pkl")
#bow_surf  = pd.read_pickle(path + "/bow_surf.pkl")
#color_hist_features  = pd.read_pickle(path + "/hist.pkl")
#labels  = pd.read_pickle(path +"/labels.pkl")

#print(color_layout_features)

features = np.hstack([color_layout_features, color_hist_features, bow_surf])
# define dataset
X, Y = features,labels

In [2]:
from sklearn.model_selection import train_test_split
from numpy import concatenate
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.50, random_state=1, stratify=Y)
# split train into labeled and unlabeled
X_train_lab, X_test_unlab, y_train_lab, y_test_unlab = train_test_split(X_train, y_train, test_size=0.50, random_state=1, stratify=y_train)
# create the training dataset input
X_train_mixed = concatenate((X_train_lab, X_test_unlab))
# create "no label" for unlabeled data
nolabel = [-1 for _ in range(len(y_test_unlab))]
# recombine training dataset labels
y_train_mixed = concatenate((y_train_lab, nolabel))

In [3]:
def learn_params(x_labeled, y_labeled):
    n = x_labeled.shape[0]
    phi = x_labeled[y_labeled == 1].shape[0] / n
    mu0 = np.sum(x_labeled[y_labeled == 0], axis=0) / x_labeled[y_labeled == 0].shape[0]
    mu1 = np.sum(x_labeled[y_labeled == 1], axis=0) / x_labeled[y_labeled == 1].shape[0]
    sigma0 = np.cov(x_labeled[y_labeled == 0].T, bias= True)
    sigma1 = np.cov(x_labeled[y_labeled == 1].T, bias=True)
    return {'phi': phi, 'mu0': mu0, 'mu1': mu1, 'sigma0': sigma0, 'sigma1': sigma1}

In [4]:
learn_params(X_train,y_train)

{'phi': 0.02665299846232701,
 'mu0': array([ 9.56560241e+02, -9.39984986e+00,  1.05556454e+02,  4.28190083e+01,
         4.47548467e+00,  1.75332105e+01,  2.04836501e+00, -1.30404858e+01,
        -3.68394000e+00,  1.79400308e+00,  2.86144578e+00, -4.33732113e+00,
        -1.14998845e+01,  1.84265369e+00,  3.91566265e-01, -5.87748733e-01,
         4.52774326e-01, -2.16495004e+00,  6.76716279e+00, -1.51643468e+00,
         1.64014117e+00,  4.50910114e-01,  1.56470521e+00, -2.26115506e+00,
        -1.55144766e-01,  2.75129380e+00, -1.93057283e+00,  3.52556660e+00,
        -1.62385811e+00, -4.72166688e+00, -1.77445000e+00,  4.23329790e+00,
        -3.65058883e+00, -1.45816686e+00,  2.24460971e-01,  1.75061031e+00,
        -5.60010384e-01,  2.33783612e+00, -6.75157844e-01, -3.59939759e+00,
         3.14937779e-01, -1.16216432e+00, -4.04251961e-01, -3.24638337e+00,
         1.53453677e+00,  1.52624466e+00, -2.50402834e-01,  5.08932591e-01,
        -9.58086508e-01, -2.34257111e-01,  3.2584463

In [5]:
from scipy import stats
def get_avg_log_likelihood(x, params):
    loglikelihood, _ = e_step(x, params)
    return np.mean(loglikelihood)

def run_em(x, params):
    avg_loglikelihoods = []
    while True:
        avg_loglikelihood = get_avg_log_likelihood(x, params)
        avg_loglikelihoods.append(avg_loglikelihood)
        if len(avg_loglikelihoods) > 2 and abs(avg_loglikelihoods[-1] - avg_loglikelihoods[-2]) < 0.0001:
            break
        params = m_step(x_unlabeled, params)
    print("\tphi: %s\n\tmu_0: %s\n\tmu_1: %s\n\tsigma_0: %s\n\tsigma_1: %s"
               % (params['phi'], params['mu0'], params['mu1'], params['sigma0'], params['sigma1']))
    _, posterior = e_step(x_unlabeled, params)
    forecasts = np.argmax(posterior, axis=1)
    return forecasts, posterior, avg_loglikelihoods

def e_step(x, params):
    np.log([stats.multivariate_normal(params["mu0"], params["sigma0"]).pdf(x),
            stats.multivariate_normal(params["mu1"], params["sigma1"]).pdf(x)])
    log_p_y_x = np.log([1-params["phi"], params["phi"]])[np.newaxis, ...] + \
                np.log([stats.multivariate_normal(params["mu0"], params["sigma0"]).pdf(x),
            stats.multivariate_normal(params["mu1"], params["sigma1"]).pdf(x)]).T
    log_p_y_x_norm = logsumexp(log_p_y_x, axis=1)
    return log_p_y_x_norm, np.exp(log_p_y_x - log_p_y_x_norm[..., np.newaxis])

In [6]:
learned_params = learn_params(X_train, y_train)
semisupervised_forecasts, semisupervised_posterior, semisupervised_loglikelihoods = run_em(X_test, learned_params)
print("total steps: ", len(semisupervised_loglikelihoods))
plt.plot(semisupervised_loglikelihoods)
plt.title("semi-supervised log likelihoods")
plt.savefig("semi-supervised.png")

LinAlgError: singular matrix

In [7]:
from __future__ import print_function
from time import time

import numpy as np

from sklearn.utils import check_array
from sklearn.utils.extmath import logsumexp
from scipy.special import comb
from sklearn.mixture.gmm import (
    GMM,
    log_multivariate_normal_density,
    distribute_covar_matrix_to_match_covariance_type,)
from sklearn.cluster import KMeans


class SSGMM(GMM):
    """Semi-supervised Gaussian Mixture Models.
    Representation of a Gaussian mixture model probability distribution,
    where some class labels are provided.
    This class allows for easy evaluation of, sampling from, and
    maximum-likelihood estimation of the parameters of a GMM distribution.
    Read more in the scikit-learn GMM documentation.
    """
    def __init__(self, n_components=1, covariance_type='diag',
                 random_state=None, tol=1e-3, min_covar=1e-3,
                 n_iter=100, n_init=1, params='wmc', init_params='wmc',
                 verbose=0):
        super(SSGMM, self).__init__(
            n_components=n_components, covariance_type=covariance_type,
            random_state=random_state, tol=tol, min_covar=min_covar,
            n_iter=n_iter, n_init=n_init, params=params,
            init_params=init_params, verbose=verbose)

    def _score_samples(self, X, y):
        """
        Calculate the log-probability for each datapoint for each class.
        In the case where labels are provided, log-probabilities for other class
        labels are set to -infinity so that they end up contributing zero to the
        likelihood.
        """

        # Firstly, calculate old unsupervised likelihood.
        lpr = (log_multivariate_normal_density(X, self.means_, self.covars_,
                                               self.covariance_type) +
               np.log(self.weights_))

        # Remove contributions of classes not matching labelled data.
        for label in self.label_classes_:
            label_indices = np.where(y == label)[0]
            label_mask = np.delete(np.arange(self.n_components), label)
            # @@@ probably better numpythonic way to do this...
            for m in label_mask:
                lpr[label_indices, m] = -np.inf

        logprob = logsumexp(lpr, axis=1)
        responsibilities = np.exp(lpr - logprob[:, np.newaxis])

        return logprob, responsibilities

    def score_samples(self, X):
        return self._score_samples(X, y=np.array([]))

    def _init_params(self, X, y):
        """
        Initialise parameters for each run of the EM algorithm.
        For classes with labels provided, means are initialised as the centroid
        of the labelled data. Remaining class means are initialised via K-Means.
        """
        if 'm' in self.init_params or not hasattr(self, 'means_'):
            # First include means of labelled data.
            self.means_ = np.zeros((self.n_components, X.shape[1]))
            for label in self.label_classes_:
                self.means_[label] = np.mean(
                    X[np.where(y == label)[0], :], axis=0)

            # Initialise remaining means (ones with no labels) using K-Means.
            blanks = np.delete(np.arange(self.n_components), self.label_classes_)
            if len(blanks) != 0:
                X_unlabelled = X[y == -1, :]

                means = KMeans(
                    n_clusters=len(blanks), random_state=self.random_state).fit(
                        X_unlabelled).cluster_centers_

                self.means_[blanks, :] = means

            if self.verbose > 1:
                print('\tMeans have been initialized.')
                print(self.means_)

        if 'w' in self.init_params or not hasattr(self, 'weights_'):
            self.weights_ = np.tile(1.0 / self.n_components,
                                    self.n_components)
            if self.verbose > 1:
                print('\tWeights have been initialized.')
                print(self.weights_)

        if 'c' in self.init_params or not hasattr(self, 'covars_'):
            cv = np.cov(X.T) + self.min_covar * np.eye(X.shape[1])
            if not cv.shape:
                cv.shape = (1, 1)
            self.covars_ = \
                distribute_covar_matrix_to_match_covariance_type(
                    cv, self.covariance_type, self.n_components)
            if self.verbose > 1:
                print('\tCovariance matrices have been initialized.')
                print(self.covars_)

    def _fit(self, X, y, do_prediction=False):
        # Initialization step
        X = check_array(X, dtype=np.float64, ensure_min_samples=2,
                        estimator=self)
        if X.shape[0] < self.n_components:
            raise ValueError(
                'GMM estimation with %s components, but got only %s samples' %
                (self.n_components, X.shape[0]))

        max_log_prob = -np.infty

        if y is None:
            self.label_classes_ = []
        else:
            labels = np.unique(y)
            self.label_classes_ = labels[labels != -1].astype(int)
            if max(labels) > self.n_components:
                raise ValueError(
                    'More labels provided than number of components.')

        if self.verbose > 0:
            print('Expectation-maximization algorithm started.')

        for init in range(self.n_init):
            # Initialize prior distribution.
            if self.verbose > 0:
                print('Initialization ' + str(init + 1))
                start_init_time = time()
            self._init_params(X, y)

            # EM algorithms
            current_log_likelihood = None
            # reset self.converged_ to False
            self.converged_ = False

            for i in range(self.n_iter):
                if self.verbose > 0:
                    print('\tEM iteration ' + str(i + 1))
                    start_iter_time = time()
                prev_log_likelihood = current_log_likelihood

                # Expectation step
                log_likelihoods, responsibilities = self._score_samples(X, y)
                current_log_likelihood = log_likelihoods.mean()

                # Check for convergence.
                if prev_log_likelihood is not None:
                    change = abs(current_log_likelihood - prev_log_likelihood)
                    if self.verbose > 1:
                        print('\t\tChange: ' + str(change))
                    if change < self.tol:
                        self.converged_ = True
                        if self.verbose > 0:
                            print('\t\tEM algorithm converged.')
                        break

                # Maximization step
                self._do_mstep(X, responsibilities, self.params,
                               self.min_covar)
                if self.verbose > 1:
                    print('\t\tEM iteration ' + str(i + 1) + ' took {0:.5f}s'.format(
                        time() - start_iter_time))

            # if the results are better, keep it
            if self.n_iter:
                if current_log_likelihood > max_log_prob:
                    max_log_prob = current_log_likelihood
                    best_params = {'weights': self.weights_,
                                   'means': self.means_,
                                   'covars': self.covars_}
                    if self.verbose > 1:
                        print('\tBetter parameters were found.')

            if self.verbose > 1:
                print('\tInitialization ' + str(init + 1) + ' took {0:.5f}s'.format(
                    time() - start_init_time))

        # check the existence of an init param that was not subject to
        # likelihood computation issue.
        if np.isneginf(max_log_prob) and self.n_iter:
            raise RuntimeError(
                "EM algorithm was never able to compute a valid likelihood " +
                "given initial parameters. Try different init parameters " +
                "(or increasing n_init) or check for degenerate data.")

        if self.n_iter:
            self.covars_ = best_params['covars']
            self.means_ = best_params['means']
            self.weights_ = best_params['weights']
        else:  # self.n_iter == 0 occurs when using GMM within HMM
            # Need to make sure that there are responsibilities to output
            # Output zeros because it was just a quick initialization
            responsibilities = np.zeros((X.shape[0], self.n_components))

        return responsibilities

    def fit(self, X, y=None):
        """
        Estimate model parameters with the EM algorithm.
        A initialization step is performed before entering the
        expectation-maximization (EM) algorithm. If you want to avoid
        this step, set the keyword argument init_params to the empty
        string '' when creating the GMM object. Likewise, if you would
        like just to do an initialization, set n_iter=0.
        Parameters
        ----------
        X : array_like, shape (n_samples, n_features)
            List of n_features-dimensional data points.  Each row
            corresponds to a single data point.
        y : array_like, shape (n_samples)
            List of labels for data points in X. Labels should be non-negative
            integers, in the range [0,...,n_components]. Unlabelled data should
            be given the label -1. If None, defaults to standard GMM.
        Returns
        -------
        self
        """
        self._fit(X, y)
        return self

ImportError: cannot import name 'logsumexp'

In [ ]:
pip install -U scikit-learn scipy statsmodels

In [ ]:
scipy.__version__

In [ ]:
pip uninstall scikit-learn

In [ ]:
pip install scikit-learn==0.18.2

In [ ]:
scikit.__version__